# Machine Learning na Robótica

Neste workshop você desenvolverá um método de aprendizagem utilizando Algoritmo Genético, um método de otimização bio-inspirado que te introduzirá ao mundo da Aprendizagem de Máquina. Contextualizado em uma aplicação de robótica móvel, você irá experienciar na prática como IA e Robótica se conectam aprendendo a modelagem e simulação de um Robô de Tração Diferencial Go-to-Goal.

In [1]:
# Importing libraries...
import time
import numpy as np
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

from GoPiGo3 import GoPiGo3

# Inicia Cliente
client = RemoteAPIClient()  # Objeto do Cliente
sim = client.getObject("sim")  # Simulation object

## Funções Fundamentais para Algoritmos Genéticos

In [2]:
# Gera uma matriz com valores aleatórios para cada parâmetro do indivíduo
def init_population(population_size:int, genome_lenght:int)->np.ndarray:
    return np.random.uniform(0.0, 3.0, (population_size, genome_lenght))

In [3]:
# Modifica aleatóriamente os pesos de forma proporcional
def mutate(genome:np.ndarray, mutation_rate:float)->np.ndarray:
    new_genome = genome * np.random.uniform(1.0 - mutation_rate, 1.0 + mutation_rate, len(genome)) # It can drop or rise by 25%
    return new_genome

# Modifica aleatoriamente os pesos de forma aditiva
def mutateAdditive(genome:np.ndarray, mutation_rate:float)->np.ndarray:
    new_genome = genome + np.random.uniform(1.0 - mutation_rate, 1.0 + mutation_rate, len(genome))
    return new_genome

# Modifica aleatoriamente os pesos de forma binaria
def mutateBinary(genome:np.ndarray, mutation_rate:float, ammount:float)->np.ndarray:
    new_genome = genome + ammount*np.random.choice(2, len(genome),p=[1.0-mutation_rate,mutation_rate])
    return new_genome

In [4]:
# Tem uma chance de misturar os genomas num ponto de corte aleatório
# Aa + Bb -> Ab + Ba
def crossover(genome_x:np.ndarray, genome_y:np.ndarray, crossover_rate:float)->tuple[np.ndarray,np.ndarray]:
    if np.random.rand() < crossover_rate:
        c = np.random.randint(0, len(genome_x) - 1)

        return np.concatenate((genome_x[:c], genome_y[c:])), np.concatenate((genome_y[:c], genome_x[c:]))
    else:
        return genome_x, genome_y


In [5]:
# Calcula de acordo com pesos passados uma pontuação para os resultados
def get_fitness(metrics:np.ndarray, weights:np.ndarray)->float:
    return np.dot(metrics, weights)

In [6]:
# Com a pontuação de cada filho seleciona novos individuos para a nova geração
def pick_fittest(population:np.ndarray, fitness_values:np.ndarray)->np.ndarray:
    return population[np.argsort(fitness_values)][-2:]

# Com a pontuação de cada filho seleciona novos individuos para a nova geração de forma ponderada
def pick_wheigted_fittest(population:np.ndarray, fitness_values:np.ndarray)->np.ndarray:
    return np.random.choice(population, 2,p=fitness_values/fitness_values.sum())

In [7]:
# Reseta individuos no CoppeliaSim
def reset_individuals(population_size):
    while True:
        try:
            sim.removeModel(sim.getObject(f"/Individual[1]"))
        except:
            break

    base_handle = sim.getObject("/Individual")
    for _ in range(population_size - 1):
        sim.copyPasteObjects([base_handle], 1)

In [ ]:
# Simulation batch
simulation_time = 5.0
target_lower_bound = 0.1
target_higher_bound = 1.0

generations = 50
population_size = 10
crossover_rate = 0.10
mutation_rate = 0.10
weights = np.array([
    -40.0, # NÃO CHEGOU NO ALVO
    -1.0,  # TEMPO QUE DEMOROU PARA CHEGAR NO ALVO
    -20.0, # DISTÂNCIA DO ALVO
    -50.0, # MÁXIMA ROLAGEM
    -50.0  # MÁXIMA GUINADA
])

population = np.repeat([[3.0, 0.001, 0.01, 1.0, 0.001, 0.01]], population_size, axis=0)
population = init_population(population_size, 6)
fitness_values = None

reset_individuals(population_size)

for generation, (x, y) in enumerate(np.random.uniform(-0.5, 0.5, (generations, 2))):
    print(f"Generation {generation}: Target at x={x:.2f} ; y={y:.2f} m")

    sim.setObjectPosition(
        sim.getObject('/Target'),
        [x, y, 0.05]
    )

    robot_population = [
        GoPiGo3(
            simulation_object=sim,
            handle_name=f"/Individual[{i}]/GoPiGo3",
            target_handle_name="/Target",
            genome=genome
        )
        
        for i, genome in enumerate(population)
    ]

    try:
        sim.setStepping(True)  # Trigger simulation step manually

        # Start simulation
        sim.startSimulation()

        while sim.getSimulationTime() < simulation_time:
            for robot in robot_population:
                
                robot.sense_and_actuate()

            sim.step() # Call for next step

        # Stop simulation
        sim.stopSimulation()
        time.sleep(1.0) # Wait for simulation to stop

    except:
        pass
    
    # Get fitnes of every individual of the population
    fitness_values = [get_fitness(robot.get_metrics(), weights) for robot in robot_population]

    for robot, fitness_value in zip(robot_population, fitness_values):
        metrics = robot.get_metrics()
        
        print(f"Not converged: {robot.not_converged}") 
        print(f"Convergence time: {robot.convergence_time:.2f}") 
        print(f"Target distance: {robot.target_distance:.2f}") 
        print(f"Max roll: {robot.max_roll:.3f}") 
        print(f"Max pitch: {robot.max_pitch:.3f}") 
        print()
        print(robot.genome)
        print()
        print(f"Fitness: {fitness_value:.2f}", '\n\n')

    # Pick fittest and have them make them have offspring
    parent_x, parent_y = pick_fittest(population, fitness_values)

    # If it is on last generation, do not crossover and mutate
    if generation >= generations - 1:
        break

    # Make a new population based on variants of the most adapted individuals 
    new_population = []
    offspring_x, offspring_y = crossover(parent_x, parent_y, crossover_rate)

    for _ in range(population_size // 2):
        new_population.append(mutate(offspring_x, mutation_rate))
        new_population.append(mutate(offspring_y, mutation_rate))

    # Update population
    population = np.array(new_population)

Generation 0: Target at x=-0.39 ; y=-0.43 m
Not converged: True
Convergence time: 5.00
Target distance: 0.64
Max roll: 3.141
Max pitch: 1.460

[2.30191936 2.92953947 2.16629344 1.95778529 2.14295333 2.59552014]

Fitness: -287.77 


Not converged: True
Convergence time: 5.00
Target distance: 0.65
Max roll: 3.140
Max pitch: 1.355

[2.03441647 2.79106062 2.65654622 1.36484594 2.88114927 2.7634177 ]

Fitness: -282.84 


Not converged: True
Convergence time: 5.00
Target distance: 0.65
Max roll: 3.142
Max pitch: 1.363

[2.44681095 2.16957134 1.72541621 1.22443171 0.05088888 1.41421528]

Fitness: -283.24 


Not converged: True
Convergence time: 5.00
Target distance: 0.11
Max roll: 0.019
Max pitch: 0.018

[1.46187771 0.06051594 1.71227557 1.8645616  1.26061652 0.00562702]

Fitness: -49.01 


Not converged: True
Convergence time: 5.00
Target distance: 0.59
Max roll: 3.142
Max pitch: 1.542

[2.82268489 2.81560641 1.16829513 2.75441296 0.7383562  2.80858479]

Fitness: -291.06 


Not converged: Tr

In [ ]:
genome = pick_fittest(population, fitness_values)[-1] # Pick fittest genome

reset_individuals(1)

robot = GoPiGo3(
    simulation_object=sim,
    handle_name="/Individual/GoPiGo3",
    target_handle_name="/Target",
    genome=genome
)

try:
    sim.setStepping(True)  # Trigger simulation step manually

    # Start simulation
    sim.startSimulation()
    
    while not sim.getSimulationStopping():
        robot.sense_and_actuate()
        sim.step()

    # Stop simulation
    sim.stopSimulation()

except:
    pass